## RDP Learning

In this notebook, we show how to perform PAC learning
over RDPs.

### Experiment 1: Rotating MAB

In [1]:
from functools import partial
from typing import Tuple, Set, Dict

import gym
import numpy as np
from gym.wrappers import TimeLimit
from notebooks.utils import render_digraph

from src import NonMarkovianRotatingMAB
from src.learn_pdfa.base import learn_pdfa, Algorithm
from src.learn_pdfa.common import MultiprocessedGenerator
from src.learn_rdps import random_exploration_policy, RDPGenerator
from src.pdfa.render import to_graphviz_from_graph
from src.pdfa.types import Character

env = NonMarkovianRotatingMAB(winning_probs=[0.9, 0.2])

print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

s = env.reset()
print(f"Initial state: {s}")

action = 0
sp, reward, _, _ = env.step(action)
print("-" * 10)
print(f"Action: {action}")
print(f"Next state: {sp}")
print(f"Reward: {reward}")

sp, reward, _, _ = env.step(action)
print("-" * 10)
print(f"Action: {action}")
print(f"Next state: {sp}")
print(f"Reward: {reward}")


Observation space: Discrete(2)
Action space: Discrete(2)
Initial state: 0
----------
Action: 0
Next state: 1
Reward: 1.0
----------
Action: 0
Next state: 1
Reward: 1.0


### Learning

In [2]:
def learning_rotating_mab(
    stop_probability: float,
    winning_probabilities: Tuple[float, ...],
    max_episode_steps: int,
    nb_samples: int,
    delta: float,
    n_upperbound: int,
    nb_processes: int = 8,
) -> Tuple[RDPGenerator, Tuple[Set[int], Dict[int, Dict[Character, int]]]]:
    """Test learning of Rotating MAB."""
    env = gym.make("NonMarkovianRotatingMAB-v0", winning_probs=winning_probabilities)
    env = TimeLimit(env, max_episode_steps=max_episode_steps)

    policy = partial(random_exploration_policy, env)

    rdp_generator = RDPGenerator(
        env, policy=policy, nb_rewards=2, stop_probability=stop_probability
    )

    examples = rdp_generator.sample(n=1000)

    print(
        f"Apriori expected length of traces: 1/stop_prob = {1 / (stop_probability + np.finfo(float).eps)}"
    )
    print(f"Average length of traces: {np.mean([len(e) for e in examples])}")

    mp_rdp_generator = MultiprocessedGenerator(rdp_generator, nb_processes=nb_processes)
    v, t = learn_pdfa(
        algorithm=Algorithm.BALLE,
        nb_samples=nb_samples,
        sample_generator=mp_rdp_generator,
        alphabet_size=rdp_generator.alphabet_size(),
        delta=delta,
        n=n_upperbound,
    )
    return rdp_generator, (v, t)

##### Rotating MAB, 2 arms, with probabilities (0.7, 0.3)

In [3]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(0.7, 0.3),
    max_episode_steps=15,
    nb_samples=100000,
    delta=0.1,
    n_upperbound=4,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))

Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.679


[2020-11-06 19:32:32,880][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7f2954dafa50>, nb_samples=100000, n=4, alphabet_size=8, '
 'delta=0.1)')
[2020-11-06 19:32:39,747][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 19:32:40,398][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 19:32:41,100][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 19:32:41,843][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 19:32:42,714][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 19:32:43,593][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 19:32:44,374][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 19:32:45,196][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 19:32:46,016][src.learn_pdfa][INFO] Iteration 8
/home/marcofavorito/.local/share/virtualenvs/PAC-RDPs-code-h40RRNYH/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 'shape' argument should be used instead of 'dims'
  # Remove the CWD from sys.path while we

##### Rotating MAB, 3 arms, with probabilities (1.0, 0.0, 0.0)

In [4]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(1.0, 0.0, 0.0),
    max_episode_steps=15,
    nb_samples=100000,
    delta=0.1,
    n_upperbound=5,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))

Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.679


[2020-11-06 19:32:52,234][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7f296b951bd0>, nb_samples=100000, n=5, alphabet_size=12, '
 'delta=0.1)')
[2020-11-06 19:32:58,055][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 19:32:58,754][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 19:32:59,415][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 19:33:00,114][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 19:33:00,992][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 19:33:01,833][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 19:33:02,645][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 19:33:03,377][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 19:33:04,129][src.learn_pdfa][INFO] Iteration 8
[2020-11-06 19:33:04,906][src.learn_pdfa][INFO] Iteration 9
/home/marcofavorito/.local/share/virtualenvs/PAC-RDPs-code-h40RRNYH/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 'shape' argument should be use

##### Rotating MAB, 3 arms, with probabilities (0.1, 0.2, 0.9)

In [11]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(0.1, 0.2, 0.9),
    max_episode_steps=1000,
    nb_samples=1000000,
    delta=0.05,
    n_upperbound=6,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))

Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.84


[2020-11-06 20:27:32,525][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7f29540e99d0>, nb_samples=500000, n=6, alphabet_size=12, '
 'delta=0.05)')
[2020-11-06 20:27:58,369][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 20:28:01,437][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 20:28:04,495][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 20:28:07,928][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 20:28:11,383][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 20:28:14,859][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 20:28:18,566][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 20:28:22,342][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 20:28:26,084][src.learn_pdfa][INFO] Iteration 8
[2020-11-06 20:28:29,905][src.learn_pdfa][INFO] Iteration 9
[2020-11-06 20:28:33,778][src.learn_pdfa][INFO] Iteration 10
[2020-11-06 20:28:37,764][src.learn_pdfa][INFO] Iteration 11
[2020-11-06 20:28:41,802][src.learn_pdfa][INFO] I

##### Rotating MAB, 4 arms, with probabilities (1.0, 0.0, 0.0, 0.0)


In [4]:
rdp_generator, (v, t) = learning_rotating_mab(
    stop_probability=0.2,
    winning_probabilities=(1.0, 0.0, 0.0, 0.0),
    max_episode_steps=50,
    nb_samples=300000,
    delta=0.1,
    n_upperbound=5,
    nb_processes=8)

render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))


Apriori expected length of traces: 1/stop_prob = 4.999999999999994
Average length of traces: 3.84


[2020-11-06 21:28:24,520][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7f1b88f7ca50>, nb_samples=300000, n=5, alphabet_size=16, '
 'delta=0.1)')
[2020-11-06 21:28:36,628][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 21:28:38,105][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 21:28:39,752][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 21:28:41,548][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 21:28:43,771][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 21:28:46,167][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 21:28:48,428][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 21:28:50,795][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 21:28:53,328][src.learn_pdfa][INFO] Iteration 8
[2020-11-06 21:28:55,860][src.learn_pdfa][INFO] Iteration 9
[2020-11-06 21:28:58,476][src.learn_pdfa][INFO] Iteration 10
[2020-11-06 21:29:01,108][src.learn_pdfa][INFO] Iteration 11
[2020-11-06 21:29:03,925][src.learn_pdfa][INFO] It

##### Rotating MAB, 5 arms, with probabilities (1.0, 0.0, 0.0, 0.0, 0.0)


In [3]:
# rdp_generator, (v, t) = learning_rotating_mab(
#     stop_probability=0.15,
#     winning_probabilities=(1.0, 0.0, 0.0, 0.0, 0.0),
#     max_episode_steps=100,
#     nb_samples=1000000,
#     delta=0.05,
#     n_upperbound=10,
#     nb_processes=6)
#
# render_digraph(to_graphviz_from_graph(v, t, char2str=lambda c: str(rdp_generator.decoder(c))))

Apriori expected length of traces: 1/stop_prob = 6.666666666666657
Average length of traces: 5.611


[2020-11-06 22:11:12,955][src.learn_pdfa][INFO] Parameters: ('BalleParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7f3400f48910>, nb_samples=1000000, n=10, alphabet_size=20, '
 'delta=0.05)')
[2020-11-06 22:12:16,380][src.learn_pdfa][INFO] Iteration 0
[2020-11-06 22:12:25,422][src.learn_pdfa][INFO] Iteration 1
[2020-11-06 22:12:35,280][src.learn_pdfa][INFO] Iteration 2
[2020-11-06 22:12:45,929][src.learn_pdfa][INFO] Iteration 3
[2020-11-06 22:12:59,253][src.learn_pdfa][INFO] Iteration 4
[2020-11-06 22:13:14,098][src.learn_pdfa][INFO] Iteration 5
[2020-11-06 22:13:28,238][src.learn_pdfa][INFO] Iteration 6
[2020-11-06 22:13:43,299][src.learn_pdfa][INFO] Iteration 7
[2020-11-06 22:13:58,909][src.learn_pdfa][INFO] Iteration 8
[2020-11-06 22:14:15,204][src.learn_pdfa][INFO] Iteration 9
[2020-11-06 22:14:32,561][src.learn_pdfa][INFO] Iteration 10
[2020-11-06 22:14:48,810][src.learn_pdfa][INFO] Iteration 11
[2020-11-06 22:15:05,389][src.learn_pdfa][INFO]